# Parsing data

In [24]:
# Imports

import pandas as pd
import numpy as np
import statsmodels as sms
import collections
import csv
import scipy as sp
from scipy import interpolate

# Variables

filenames = [
    "genes_CSP_I7_C1",
    "genes_CSP_RTS_CONTROL",
    "genes_iRBC_I7_C1",
    "genes_iRBC_RTS_CONTROL",
    "genes_malaria_CSP_CONTROL",
    "genes_malaria_CSP_RTS",
    "genes_malaria_iRBC_CONTROL",
    "genes_malaria_iRBC_RTS",
    "genes_protected_CSP_C1",
    "genes_protected_CSP_I7",
    "genes_protected_iRBC_C1",
    "genes_protected_iRBC_I7"
]

## Obtain a mapping file from probeset ids to gene symbols

In [25]:
D = pd.read_csv("data/HuGene-2_1-st-v1.na36.hg19.probeset.csv", skiprows = 22)


In [26]:
probesets   = np.array(D["probeset_id"]) 
clusts      = np.array(D["transcript_cluster_id"])
assigns     = np.array(D["gene_assignment"])
rna_assigns = np.array(D["mrna_assignment"])

In [44]:
missing_transcripts = []
for p, c, ass, rna in zip(probes, clusts, assigns, rna_assigns):
    for x in ass.split(" /// "):
        if x == "---":
            if rna == "---": continue
            for y in rna.split(" /// "):
                missing_transcripts += [y.split(" // ")[0]]
missing_transcripts = set(missing_transcripts)

In [47]:
# Run a biomaRt script to map these files

library(biomaRt)


chdir("~/Desktop/")
ensembl = useMart("ENSEMBL_MART_ENSEMBL",dataset="hsapiens_gene_ensembl", host = "www.ensembl.org")
listFilters(ensembl)
listAttributes(ensembl)


values = read.table("~/Desktop/missing_transcripts.tsv", header = F)$V1

ensmap <- getBM(attributes=c("ensembl_transcript_id", "hgnc_symbol"), filters = "ensembl_transcript_id", values = values, mart= ensembl)

emblmap <- getBM(attributes=c("embl", "hgnc_symbol"), filter = "embl", values = values, mart = ensembl)

write.table(ensmap, file = "~/Desktop/ensemblmap.tsv", sep = "\t", col.names = T, row.names = F, quote = F)
write.table(emblmap, file = "~/Desktop/emblmap.tsv", sep = "\t", col.names = T, row.names = F, quote = F)

with open("data/missing_transcripts.tsv", "w") as f:
    for x in sorted(missing_transcripts):
        f.write("%s\n" % x)

In [6]:
# Read these files

ensmap = pd.read_csv("data/ensemblmap.tsv", delimiter = "\t")
emblmap = pd.read_csv("data/emblmap.tsv", delimiter = "\t")

In [7]:
mrna2gene = collections.defaultdict(set)
for r in zip(np.array(ensmap["ensembl_transcript_id"]), np.array(ensmap["hgnc_symbol"])):
    if type(r[1]) is not str: continue
    mrna2gene[r[0]].update([r[1]])
for r in zip(np.array(emblmap["embl"]), np.array(emblmap["hgnc_symbol"])):
    if type(r[1]) is not str: continue
    mrna2gene[r[0]].update([r[1]])
    

In [9]:
probe2gene = collections.defaultdict(set)
missing_transcripts = []
with open("data/probeset2genesymbol.tsv", "w") as f:
    for p, c, ass, rna in zip(probesets, clusts, assigns, rna_assigns):
        for x in ass.split(" /// "):
            if x == "---":
                if rna == "---": continue
                for y in rna.split(" /// "):
                    t = y.split(" // ")[0]
                    if t in mrna2gene:
                        gs = list(mrna2gene[t])
                        for g in gs:
                            probe2gene[p].update([g])
                            probe2gene[c].update([g])
            else:
                g = x.split(" // ")[1]
                probe2gene[p].update([g])
                probe2gene[c].update([g])
    for k,v in probe2gene.iteritems():
        for x in v:
            f.write("%s\t%s\n" % (k, x))

## Probeset files and calculate FDR

In [10]:
def qvalue(pv, m=None, verbose=False, lowmem=False, pi0=None):
    """
    Estimates q-values from p-values
    Args
    =====
    m: number of tests. If not specified m = pv.size
    verbose: print verbose messages? (default False)
    lowmem: use memory-efficient in-place algorithm
    pi0: if None, it's estimated as suggested in Storey and Tibshirani, 2003.
         For most GWAS this is not necessary, since pi0 is extremely likely to be
         1
    """
    assert(pv.min() >= 0 and pv.max() <= 1), "p-values should be between 0 and 1"

    original_shape = pv.shape
    pv = pv.ravel()  # flattens the array in place, more efficient than flatten()

    if m is None:
        m = float(len(pv))
    else:
        # the user has supplied an m
        m *= 1.0

    # if the number of hypotheses is small, just set pi0 to 1
    if len(pv) < 100 and pi0 is None:
        pi0 = 1.0
    elif pi0 is not None:
        pi0 = pi0
    else:
        # evaluate pi0 for different lambdas
        pi0 = []
        lam = sp.arange(0, 0.90, 0.01)
        counts = sp.array([(pv > i).sum() for i in sp.arange(0, 0.9, 0.01)])
        for l in range(len(lam)):
            pi0.append(counts[l]/(m*(1-lam[l])))

        pi0 = sp.array(pi0)

        # fit natural cubic spline
        tck = interpolate.splrep(lam, pi0, k=3)
        pi0 = interpolate.splev(lam[-1], tck)
        if verbose:
            print("qvalues pi0=%.3f, estimated proportion of null features " % pi0)

        if pi0 > 1:
            if verbose:
                print("got pi0 > 1 (%.3f) while estimating qvalues, setting it to 1" % pi0)
            pi0 = 1.0

    assert(pi0 >= 0 and pi0 <= 1), "pi0 is not between 0 and 1: %f" % pi0

    if lowmem:
        # low memory version, only uses 1 pv and 1 qv matrices
        qv = sp.zeros((len(pv),))
        last_pv = pv.argmax()
        qv[last_pv] = (pi0*pv[last_pv]*m)/float(m)
        pv[last_pv] = -sp.inf
        prev_qv = last_pv
        for i in xrange(int(len(pv))-2, -1, -1):
            cur_max = pv.argmax()
            qv_i = (pi0*m*pv[cur_max]/float(i+1))
            pv[cur_max] = -sp.inf
            qv_i1 = prev_qv
            qv[cur_max] = min(qv_i, qv_i1)
            prev_qv = qv[cur_max]

    else:
        p_ordered = sp.argsort(pv)
        pv = pv[p_ordered]
        qv = pi0 * m/len(pv) * pv
        qv[-1] = min(qv[-1], 1.0)

        for i in xrange(len(pv)-2, -1, -1):
            qv[i] = min(pi0*m*pv[i]/(i+1.0), qv[i+1])

        # reorder qvalues
        qv_temp = qv.copy()
        qv = sp.zeros_like(qv)
        qv[p_ordered] = qv_temp

    # reshape qvalues
    qv = qv.reshape(original_shape)

    return qv

In [11]:
for filename in filenames:

    with open("data/diff_expr/"+ filename + ".txt", "r") as f:
        f.next()
        R = []
        probesets = []
        for r in csv.reader(f, delimiter = " "):
            probesets += [r[0]]
            R += [[float(r[1]), float(r[2])]]
    R = np.array(R)
    
    q = qvalue(R[:,1])
    
    with open("data/diff_expr/" + filename + ".tsv", "w") as f:
        
        for i,p in enumerate(probesets):
            
            f.write("%s\t%.5f\t%.5E\t%.5E\n" % (p, R[i,0], R[i,1], q[i]))
            

## Map to genes

Use the median to aggregate

In [27]:
stats = {}

for filename in filenames:

    with open("data/diff_expr/" + filename + ".tsv", "r") as f:
        
        ps = collections.defaultdict(list)
        adjps = collections.defaultdict(list)
        
        fcs = collections.defaultdict(list)

        for r in csv.reader(f, delimiter = "\t"):
                
            for g in probe2gene[int(r[0])]:

                fcs[g]   += [float(r[1])]
                ps[g]    += [float(r[2])]
                adjps[g] += [float(r[3])]
        
        fcs = dict((k, np.median(v)) for k,v in fcs.iteritems())
        ps = dict((k, np.median(v)) for k,v in ps.iteritems())
        adjps = dict((k, np.median(v)) for k,v in adjps.iteritems())
        
    
    with open("data/diff_expr/" + filename + ".rnk", "w") as f:
                
        for w in sorted(fcs, key=fcs.get, reverse=True):
            f.write("%s\t%.5f\t%.3E\t%.3E\n" % (w, fcs[w], ps[w], adjps[w]))
    
    # Some stats
        
    P, AP = 0, 0
    
    for k,v in ps.iteritems():
        if v <= 0.05: P += 1
    for k,v in adjps.iteritems():
        if v <= 0.25: AP += 1
              
    stats[filename] = (len(ps), P, AP)
    
with open("data/stats.tsv", "w") as f:
    
    f.write("filename\tgenes\tp0.05\tadjp0.25\n")
    for k,v in stats.iteritems():
        f.write("%s\t%d\t%d\t%d\n" % (k, v[0], v[1], v[2]))

[0.39116]
[0.58188]
[0.28763]
[-0.36869]
[0.16123]
[0.1178]
